### LangGraph Prototype-Researcher & Blog Writer

In [ ]:
pip install langchain

In [ ]:
pip install langchain-openai

In [ ]:
pip install langchain-community

In [ ]:
pip install langgraph

In [ ]:
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_tool_calling_agent
from langchain.tools import tool

# Define tools
@tool

def web_research(topic: str) -> str:
    """Fetches research information about the given topic."""
    return f"Research info about: {topic} (stubbed)"

# Agents
llm = ChatOpenAI(model="gpt-4")
research_agent = create_tool_calling_agent(llm, [web_research])
writer_agent = create_tool_calling_agent(llm, [])

research_executor = AgentExecutor(agent=research_agent, tools=[web_research], verbose=True)
writer_executor = AgentExecutor(agent=writer_agent, tools=[], verbose=True)

# Define states
def research_node(state):
    topic = state["topic"]
    research = research_executor.invoke({"input": f"Research about {topic}"})
    return {"topic": topic, "research": research["output"]}

def writing_node(state):
    research = state["research"]
    output = writer_executor.invoke({"input": f"Create a blog outline using this research: {research}"})
    return {"output": output["output"]}

# Graph
builder = StateGraph()
builder.add_node("research", research_node)
builder.add_node("write", writing_node)

builder.set_entry_point("research")
builder.add_edge("research", "write")
builder.set_finish_point("write")

graph = builder.compile()
result = graph.invoke({"topic": "AI in education"})
print(result["output"])
# Output the result
if __name__ == "__main__":
    print("Blog outline generation complete.")
    print(result["output"])